In [1]:
# !pip install ragas langchain_google_vertexai --upgrade
# !pip install datasets s3fs --upgrade

In [2]:
from langchain_google_vertexai import ChatVertexAI, VertexAIEmbeddings
from ragas.metrics import (
    context_precision,
    faithfulness,
    answer_relevancy,
    context_recall,
    answer_similarity,
    answer_correctness,
)
# from ragas.metrics.critique import harmfulness

# list of metrics we're going to use
metrics = [
    faithfulness,
    context_precision,
    answer_relevancy,
    context_recall,
    # harmfulness,
    answer_similarity,
    answer_correctness,
]

/home/watson_chua/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import json
import pandas as pd

with open('/home/watson_chua/efs/axolotl/data/predictions/gpt4_llama3_gemma2_evaluation.jsonl', 'r') as f:
    lines = f.readlines()

data = [json.loads(l) for l in lines]
df_eval = pd.DataFrame(data)


In [4]:
df_eval

,gemini_flash_evaluation,status,reference_answer,summary_points,points,title,subtitle,question_speaker,answer_speaker,question,answer,filename,hypothetical_document,date,gpt4_answer_by_hy_doc,llama3_answer,gemma2_answer
0,"```json\n{""winner"": ""b"", ""reason"": ""Answer B i...",answered,,"[101,464 non-voters from the 2023 Presidential...","101,464 non-voters from the 2023 Presidential ...",Applications to Restore Names to Registers of ...,None,Mr Leong Mun Wai,Mr Chan Chun Sing (for the Prime Minister),Mr Leong Mun Wai asked the Prime Minister (a) ...,Mr Chan Chun Sing (for the Prime Minister): As...,sitting_2024-02-29,**Title: Applications to Restore Names to Regi...,2024-02-29 00:00:00,**Reply:**\n\n**Title: Applications to Restore...,"As at end-January 2024, there were 101,464 per...",Since the close of Nomination Day for the 2023...
1,"{""winner"": ""b"", ""reason"": ""Answer B is the mos...",answered,,"[From 2021 to 2023, over 23,300 companies obta...","From 2021 to 2023, over 23,300 companies obtai...",Applications under Enterprise Financing Scheme,None,Mr Liang Eng Hwa,Mr Gan Kim Yong,Mr Liang Eng Hwa asked the Minister for Trade ...,"Mr Gan Kim Yong: From 2021 to 2023, over 23,30...",sitting_2024-02-29,**Title: Applications under Enterprise Financi...,2024-02-29 00:00:00,**Title: Applications under Enterprise Financi...,"From 2021 to 2023, the Enterprise Financing Sc...","Between 2021 and 2023, more than 23,300 compan..."
2,"{""winner"": ""c"", ""reason"": ""Answer c) is the mo...",answered,,"[About 36,400 companies have received support ...","About 36,400 companies have received support u...",Applications for Productivity Solutions Grant ...,None,Mr Liang Eng Hwa,Mr Gan Kim Yong,Mr Liang Eng Hwa asked the Minister for Trade ...,"Mr Gan Kim Yong: In the last three years, abou...",sitting_2024-02-29,### Applications for Productivity Solutions Gr...,2024-02-29 00:00:00,**Reply to Parliamentary Question on Applicati...,"In the last three years, almost 36,400 compani...","In the last three years, around 36,400 compani..."
3,"{""winner"": ""a"", ""reason"": ""Answer a) is the be...",answered,,[Schools have autonomy to decide the end times...,Schools have autonomy to decide the end times ...,Time Guidelines for Schools to Cease Co-curric...,None,Mr Leong Mun Wai,Mr Chan Chun Sing,Mr Leong Mun Wai asked the Minister for Educat...,Mr Chan Chun Sing: Schools have the autonomy t...,sitting_2024-02-29,**Title: Time Guidelines for Schools to Cease ...,2024-02-29 00:00:00,**Reply to Parliamentary Question on Time Guid...,Schools have the autonomy to determine when th...,Schools have the autonomy to decide when to st...
4,"{""winner"": ""a"", ""reason"": ""Answer a) provides ...",answered,,[Rail operators have improved track design and...,Rail operators have improved track design and ...,Mitigation Measures to Reduce Noise from Railw...,None,Mr Leong Mun Wai,Mr Chee Hong Tat,Mr Leong Mun Wai asked the Minister for Transp...,"Over the years, rail operators have improved t...",sitting_2024-02-29,**Report: Mitigation Measures to Reduce Noise ...,2024-02-29 00:00:00,**Reply to Parliamentary Question on Mitigatio...,"In addition, rail operators have continued to ...",To minimise noise disturbance arising from MRT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,"{""winner"": ""a"", ""reason"": ""Answer A is the bes...",answered,,[PUB continuously adjusts the intensity of bio...,PUB continuously adjusts the intensity of biom...,Efficacy of PUB's Biomanipulation and Other Me...,None,Mr Leong Mun Wai,Ms Grace Fu Hai Yien,Mr Leong Mun Wai asked the Minister for Sustai...,The intensity of these measures is continuousl...,sitting_2024-03-01,# Report: Efficacy of PUB's Biomanipulation an...,2024-03-01 00:00:00,**Parliamentary Reply: Efficacy of PUB's Bioma...,"Currently, we are still studying our options f...",We continue to study potential pathways for th...
299,"{""winner"": ""a"", ""reason"": ""Answer a) is th

In [5]:
from ragas import evaluate
import nest_asyncio  # CHECK NOTES
nest_asyncio.apply()


In [21]:
import google.auth
from langchain_google_vertexai import ChatVertexAI, VertexAIEmbeddings
from ragas.llms import LangchainLLMWrapper

config = {
    "project_id": "gvt0031-gcp-152-govtext-ds",
    # "chat_model_id": "gemini-1.5-flash-001",
    "chat_model_id": "gemini-1.5-pro-001",

    "embedding_model_id": "textembedding-gecko",
}

# authenticate to GCP
creds, _ = google.auth.default(quota_project_id=config["project_id"])

# create Langchain LLM and Embeddings
vertextai_llm = ChatVertexAI(
    credentials=creds,
    model_name=config["chat_model_id"],
)
vertextai_embeddings = VertexAIEmbeddings(
    credentials=creds, model_name=config["embedding_model_id"]
)

In [7]:
from datasets import Dataset

def create_dataset(answer_key):
    df_temp = df_eval[["question", answer_key, "hypothetical_document", "answer"]] 
    df_temp["contexts"] = df_eval.apply(lambda x: [x["hypothetical_document"]], axis=1)
    df_temp.rename(columns={answer_key: "answer", "answer": "ground_truth"}, inplace=True)

    ds = Dataset.from_pandas(df_temp, split="eval")

    return ds

In [8]:
dataset_gpt4 = create_dataset("gpt4_answer_by_hy_doc")
dataset_llama3 = create_dataset("llama3_answer")
dataset_gemma2 = create_dataset("gemma2_answer")

/tmp/ipykernel_691118/2950712411.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp["contexts"] = df_eval.apply(lambda x: [x["hypothetical_document"]], axis=1)
/tmp/ipykernel_691118/2950712411.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp.rename(columns={answer_key: "answer", "answer": "ground_truth"}, inplace=True)
/tmp/ipykernel_691118/2950712411.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [22]:
gpt4_result = evaluate(
    dataset_gpt4.select(range(3)),  # using 1 as example due to quota constrains
    metrics=metrics,
    llm=vertextai_llm,
    embeddings=vertextai_embeddings,
)

llama3_result = evaluate(
    dataset_llama3.select(range(3)),  # using 1 as example due to quota constrains
    metrics=metrics,
    llm=vertextai_llm,
    embeddings=vertextai_embeddings,
)

gemma2_result = evaluate(
    dataset_gemma2.select(range(3)),  # using 1 as example due to quota constrains
    metrics=metrics,
    llm=vertextai_llm,
    embeddings=vertextai_embeddings,
)


Evaluating:   0%|          | 0/18 [00:00<?, ?it/s]

Gapic client context issue detected.This can occur due to parallelization.
Gapic client context issue detected.This can occur due to parallelization.


Evaluating:   6%|▌         | 1/18 [00:01<00:26,  1.55s/it]

Gapic client context issue detected.This can occur due to parallelization.


Evaluating:  11%|█         | 2/18 [00:02<00:17,  1.11s/it]

...

Evaluating:  61%|██████    | 11/18 [00:07<00:04,  1.63it/s]Failed to parse output. Returning None.
Failed to parse output. Returning None.
Evaluating:  72%|███████▏  | 13/18 [00:07<00:02,  2.14it/s]Failed to parse output. Returning None.
Failed to parse output. Returning None.
Evaluating: 100%|██████████| 18/18 [00:14<00:00,  1.26it/s]
/home/watson_chua/.local/lib/python3.8/site-packages/ragas/evaluation.py:304: RuntimeWarning: Mean of empty slice
  value = np.nanmean(self.scores[cn])
Evaluating:   6%|▌         | 1/18 [00:00<00:09,  1.77it/s]

Gapic client context issue detected.This can occur due to parallelization.
Gapic client context issue detected.This can occur due to parallelization.


Evaluating:  56%|█████▌    | 10/18 [00:03<00:02,  3.54it/s]Retrying langchain_google_vertexai.chat_models._acompletion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/generate_content_requests_per_minute_per_project_per_base_model with base model: gemini-1.5-pro. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai..
Failed to parse output. Returning None.
Evaluating:  61%|██████    | 11/18 [00:06<00:05,  1.20it/s]Failed to parse output. Returning None.
Retrying langchain_google_vertexai.chat_models._acompletion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/generate_content_requests_per_minute_per_project_per_base_model with base model: gemini-1.5-pro. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generativ

KeyboardInterrupt: 

Retrying langchain_google_vertexai.chat_models._acompletion_with_retry.<locals>._completion_with_retry_inner in 10.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/generate_content_requests_per_minute_per_project_per_base_model with base model: gemini-1.5-pro. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai..
Retrying langchain_google_vertexai.chat_models._acompletion_with_retry.<locals>._completion_with_retry_inner in 10.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/generate_content_requests_per_minute_per_project_per_base_model with base model: gemini-1.5-pro. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai..
Exception in callback PollerCompletionQueue._handle_events(<_UnixSelecto...e debug=False>)()
handle: <Handle PollerCompletionQueue._handle_events(<_UnixSelecto...e debug=False>

In [20]:
gpt4_result.to_pandas()

,question,answer,hypothetical_document,ground_truth,contexts,faithfulness,context_precision,answer_relevancy,context_recall,answer_similarity,answer_correctness
0,Mr Leong Mun Wai asked the Prime Minister (a) ...,**Reply:**\n\n**Title: Applications to Restore...,**Title: Applications to Restore Names to Regi...,Mr Chan Chun Sing (for the Prime Minister): As...,[**Title: Applications to Restore Names to Reg...,0.5,1.0,0.882967,1.0,0.878521,0.219630
1,Mr Liang Eng Hwa asked the Minister for Trade ...,**Title: Applications under Enterprise Financi...,**Title: Applications under Enterprise Financi...,"Mr Gan Kim Yong: From 2021 to 2023, over 23,30...",[**Title: Applications under Enterprise Financ...,NaN,0.0,0.000000,NaN,0.860286,0.215071
2,Mr Liang Eng Hwa asked the Minister for Trade ...,**Reply to Parliamentary Question on Applicati...,### Applications for Productivity Solutions Gr...,"Mr Gan Kim Yong: In the last three years, abou...",[### Applications for Productivity Solutions G...,NaN,1.0,0.844345,1.0,0.882163,NaN


In [23]:
llama3_result.to_pandas()

,question,answer,hypothetical_document,ground_truth,contexts,faithfulness,context_precision,answer_relevancy,context_recall,answer_similarity,answer_correctness
0,Mr Leong Mun Wai asked the Prime Minister (a) ...,"As at end-January 2024, there were 101,464 per...",**Title: Applications to Restore Names to Regi...,Mr Chan Chun Sing (for the Prime Minister): As...,[**Title: Applications to Restore Names to Reg...,NaN,1.0,0.823216,1.0,0.872614,NaN
1,Mr Liang Eng Hwa asked the Minister for Trade ...,"From 2021 to 2023, the Enterprise Financing Sc...",**Title: Applications under Enterprise Financi...,"Mr Gan Kim Yong: From 2021 to 2023, over 23,30...",[**Title: Applications under Enterprise Financ...,NaN,0.0,0.000000,NaN,0.938607,NaN
2,Mr Liang Eng Hwa asked the Minister for Trade ...,"In the last three years, almost 36,400 compani...",### Applications for Productivity Solutions Gr...,"Mr Gan Kim Yong: In the last three years, abou...",[### Applications for Productivity Solutions G...,NaN,1.0,0.798031,1.0,0.879084,NaN


In [18]:
gemma2_result.to_pandas()

{'faithfulness': 1.0000, 'context_precision': 0.6667, 'answer_relevancy': 0.8582, 'context_recall': 1.0000, 'answer_similarity': 0.8986, 'answer_correctness': 0.4749}

In [30]:
faithfulness_prompt_template = """Give the following contexts, separated by ######################### and an answer, do the following:
1. Break down the contexts into a list of statements
2. Break down the answer into a list of statements
3. Match the two list of statements and return the following in JSON with the following keys: "statements_in_both", "statements_in_contexts_only", "statements_in_answer_only"

Return your answer as a JSON string in a JSON parsable format without any other output.

Here are the contexts:
{context}


This is the answer:
{answer}
"""

In [31]:
test_prompt = faithfulness_prompt_template.format(context="#########################".join(dataset_llama3[0]['contexts']), answer=dataset_llama3[0]['answer'])
response = vertextai_llm.invoke(test_prompt)
response

Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ServiceUnavailable: 503 recvmsg:Connection reset by peer.


AIMessage(content='```json\n{\n  "statements_in_both": [\n    "As of the end of January 2024, a total of 101,464 non-voters from the 2023 Presidential Election have successfully applied to restore their names to the Registers of Electors.",\n    "All 101,464 applications were meticulously reviewed by the Elections Department.",\n    "Upon successful verification, all applications were approved, and the names of the applicants were restored to the Registers of Electors."\n  ],\n  "statements_in_contexts_only', response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 

In [32]:
response.content

'```json\n{\n  "statements_in_both": [\n    "As of the end of January 2024, a total of 101,464 non-voters from the 2023 Presidential Election have successfully applied to restore their names to the Registers of Electors.",\n    "All 101,464 applications were meticulously reviewed by the Elections Department.",\n    "Upon successful verification, all applications were approved, and the names of the applicants were restored to the Registers of Electors."\n  ],\n  "statements_in_contexts_only'